In [1]:
from drivecycle import drivecycle, trajectory
from drivecycle.utils import utils, plots

import matplotlib.pyplot as plt
import networkx as nx

import requests
import polyline
import json
import pandas as pd
import numpy as np

In [4]:
onestop_id = "r-c3nf-1-702631-be6946"
api_key = "SN0N9PZyDIDrOf9tcm3DS1t8sbqZi1Ul"

url = f"http://transit.land/api/v1/route_stop_patterns.geojson?onestop_id={onestop_id}&api_key={api_key}"

print(url)

r = requests.get(url).json()

stops = r["features"][0]["properties"]["stop_distances"]
coords = [tuple((i[1],i[0])) for i in r["features"][0]["geometry"]["coordinates"]]

b = polyline.encode(coords, 6)

url_trace = "http://localhost:8002/trace_attributes" 
data =json.dumps({"encoded_polyline":b,"costing":"auto","filters":{"attributes":["edge.way_id","edge.names","edge.length","edge.speed","node.intersecting_edge.road_class","node.intersecting_edge.begin_heading","node.elapsed_time","node.type"],"action":"include"}})

trace = requests.post(url_trace, data=data)
edges = trace.json()["edges"]

# edges

http://transit.land/api/v1/route_stop_patterns.geojson?onestop_id=r-c3nf-1-702631-be6946&api_key=SN0N9PZyDIDrOf9tcm3DS1t8sbqZi1Ul


In [7]:
stops = [i for i in stops if i!=0]


NameError: name 'pickle' is not defined

In [ ]:
data_ = []
for edge in edges:
    length = edge["length"]*1000
    speed = edge["speed"]
    way_id = edge["way_id"]
    try:
        intersect = [edge["end_node"]["intersecting_edges"][0]["road_class"]]
    except:
        intersect = [None]

    item = {
        "way_id":way_id,
        "length":length,
        "speed":speed,
        "intersection":intersect,
    }
    data_.append(item)

In [ ]:
data_

In [ ]:
a = utils.Graph(data_)
c = a.get_source_target()
b = utils.include_stops(a.simplify_graph(),stops) 

simple_edges = utils.get_edges(b,c)

In [ ]:
simple_edges = [i for i in simple_edges if i["length"]>10]

In [ ]:
simple_edges

In [ ]:
stop={"bus_stop":30,"tertiary":10}
dc = drivecycle.Drivecycle(simple_edges,stops=stop, stop_at_node=True, step=1)
traj = dc.get_trajectory()

In [ ]:
plots.plot_vd(traj,"test.png")